참고: https://flonelin.wordpress.com/2024/05/15/ragretrieval-augmented-generation-%ED%8F%89%EA%B0%80-%EB%B0%A9%EB%B2%95/

In [10]:
###### source document 준비 ######

In [6]:
import fitz  # PyMuPDF
from tqdm import tqdm

# PDF 파일에서 텍스트를 추출하는 함수
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    texts = []
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text = page.get_text()
        texts.append({"text": text, "source": f"page_{page_num + 1}"})
    return texts

# PDF 파일 경로
pdf_path = r"D:\[24]ICT_Practice\practice_file\(2024)포트미스_가이드북.pdf"

# PDF 파일에서 텍스트 추출
ds = extract_text_from_pdf(pdf_path)

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document as LangchainDocument
 
# 데이터셋에서 각 문서를 처리하여 LangchainDocument 객체 리스트로 생성
langchain_docs = [LangchainDocument(page_content=doc["text"], metadata={"source": doc["source"]}) for doc in tqdm(ds)]
 
#  문서를 특정 크기의 청크로 분할하기 위한 설정
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=200,
    add_start_index=True,
    separators=["\n\n", "\n", ".", " ", ""],
)
 
# 모든 문서를 분할하여 처리된 문서 리스트에 추가
docs_processed = []
for doc in langchain_docs:
    docs_processed += text_splitter.split_documents([doc])

100%|███████████████████████████████████████████████████████████████████████████████| 43/43 [00:00<00:00, 84992.97it/s]


In [8]:
# 결과 확인
print(docs_processed)

[Document(metadata={'source': 'page_1', 'start_index': 0}, page_content='가이드북\n글로벌 리더로 도약하기 위한'), Document(metadata={'source': 'page_2', 'start_index': 0}, page_content='글로벌 물류 중심\n대한민국\n해운산업\n리더국가\n고객과 \n함께하는 \n미래\n해운업 \n상생을 통한\n해운강국 건설\n가이드북\n글로벌 리더로 도약하기 위한\nCONTENTS\n세계 해운산업 리더국가 실현\n우리나라 항만을 글로벌 물류 중심지로\n해운, 조선 상생을 통한 해운강국 건설\n04\n06\n08\n10\n12\n14\n16\n18\n20\n22\n24\n26\n28\n포트미스 주요 서비스\n포트미스 이용 환경\n포트미스 회원가입 절차\n입·출항 선박에 대한 선박제원 등록\n(변경) 절차 업무 흐름\n외항선 민원신고 절차\n내항선 민원신고 절차\n외항선입항(출항)신고(최초,변경)\n승무원/승객명부\n강제도선 면제신청\n예선사용 면제신청\n선박평형수신고\n선박보안정보통보신고\n저속운항선박지원신청\n저속운항선박 인센티브 신청\n항만시설사용허가신청서(선석신청)\n항만시설사용료(정박료)면제신청\n30\n32\n34\n36\n38\n40\n42\n44\n48\n58\n80\n위험물 반입신고 /\n위험물 일람표신고\n통합화물신고\n외항입항(출항)신고(최종)\n항만시설사용신고(화물료)\n화물료 대납경비 청구\n국가관리연안항신고 및\n연안항 화물료 신고\n서북도서출항신고\n월정료선박사용신청\n선박수리신고 신청 /\n선박수리허가 신청\n상계/환급 신청\n포트미스 히스토리\n부록 신고서식\n항만시설 사용료 디지털 방식 \n03\n02 • PORT-MIS'), Document(metadata={'source': 'page_3', 'start_index': 0}, page_content='해운산업 리더국가 실현\n글로벌 경갱력을 갖춘 고부가가치 스마트 항만 실현\n우리

In [23]:
# 전체 문서를 하나의 큰 텍스트 블록으로 결합
combined_text = " ".join([doc.page_content for doc in docs_processed])

In [11]:
###### 질문 생성을 위한 agent setup ######

In [21]:
import openai
import os

# OpenAI API 키 설정 (환경 변수로 설정하는 것이 일반적)
openai.api_key = "개인키입력"

# GPT-4를 호출하는 함수 정의
def call_llm(prompt, model="gpt-3.5-turbo", temperature=0):
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=1000
    )
    answer = response.choices[0].message.content
    return answer

# 함수를 호출하여 테스트 프롬프트로 모델을 실행하고 출력 결과 확인
print(call_llm("This is a test context"))

Great! How can I assist you with your test context today?


In [22]:
QA_generation_prompt = """
당신의 임무는 주어진 맥락을 바탕으로 사실적인 질문과 답변을 작성하는 것입니다.
당신의 사실적 질문은 맥락에서 구체적이고 간결한 사실 정보로 답변될 수 있어야 합니다.
사실적 질문은 사용자가 검색 엔진에 물을 수 있는 방식으로 구성되어야 합니다.
이는 사실적 질문이 "해당 글에 따르면"이나 "맥락에 따르면"과 같은 표현을 포함해서는 안 된다는 것을 의미합니다.
 
다음과 같이 답변을 제공하세요:
 
Output:::
Factoid question: (당신의 사실적 질문)
Answer: (사실적 질문에 대한 답변)
 
이제 여기에 맥락이 있습니다.
 
Context: {context}\n
Output:::"""

In [24]:
import random
 
N_GENERATIONS = 50  # QA 쌍 50개 생성

print(f"Generating {N_GENERATIONS} QA couples...")

outputs = []
for _ in tqdm(range(N_GENERATIONS)):
    # 임의로 청크를 선택
    sampled_context = random.choice(docs_processed)
    # QA 쌍 생성
    output_QA_couple = call_llm(QA_generation_prompt.format(context=sampled_context.page_content))
    try:
        # 생성된 텍스트에서 질문과 답변을 추출
        question = output_QA_couple.split("Factoid question: ")[-1].split("Answer: ")[0].strip()
        answer = output_QA_couple.split("Answer: ")[-1].strip()
        assert len(answer) < 300, "Answer is too long"  # 답변 길이 제한 검사

        outputs.append(
            {
                "context": sampled_context.page_content,  # 문맥
                "question": question,  # 질문
                "answer": answer,  # 답변
                "source_doc": sampled_context.metadata["source"],  # 문서 출처 정보
            }
        )
    except:
        continue

Generating 50 QA couples...


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [01:18<00:00,  1.58s/it]


In [25]:
# 결과 출력
for output in outputs:
    print(f"Context: {output['context']}")
    print(f"Question: {output['question']}")
    print(f"Answer: {output['answer']}")
    print(f"Source: {output['source_doc']}\n")

Context: ▶ 위험물 반입신고 / 위험물일람표 신고
- 
위험물을 무역항의 수상구역등으로 반입시 반입 24시간 전에 신고
- 
다만, 위험물을 육상으로 반입하는 경우에는 무역항의 육상구역으로 위험물을 들여오기 전까지, 전(前) 출항지로부터 
반입 항까지의 운항 시간이 24시간 이내이고, 해상으로 위험물을 반입하는 경우에는 무역항의 수상구역등으로 위험
물을 들여오기 전까지 신고
선박입출항법 제32조1항, 시행규칙 제14조 1항
근거
법령
(담당자)수리
 • 
업무담당자: 민원 신고문서 심사후 이상이 없으면 수리
처리
7
신고 문서검토 
및 저장
• 신고내역 확인 및 저장
• 담당자 반려시 반려에 대한 내용 확인 후 재신고
4
수리
7
민원 신고
완료
8
• 위험물 반입신고 / 위험물 일람표 신고 완료
입항신고
• 선박 입항 전까지 입항 예정정보를 신고(최초)
2
위험물 신고
• 
선박에 싣고있는 위험물질 품목 및 중량, 위험물질 등급, 
하역장소 등을 신고
• 위험물 반입신고서, 위험물 일람표 신고
3
오류통보(반려)
6
• 
업무담당자 : 민원 신고서에 오류가 있는 경우 오류내용 
입력 후 (반려)처리
• 
민원인 : 반려 내용 확인 후 신고서를 수정하여 재신고
(담당자)민원 
신고문서 확인
(수리/반려)
• 
업무담당자는 민원 신고 문서 심사 후 수리 or 오류
통보(반려)처리
• 
반려시에는 민원인이 반려 내용을 확인할 수 있도록 
반려 사유를 입력
5
8  위험물 반입신고 / 위험물일람표 신고
가. 신고 업무 흐름도
나. 신고 업무 흐름도 설명
신고 프로그램 
접속
• 해양수산부(포트미스) : www.portmis.go.kr
    
PORT-MIS신고 → 위험물 → 위험물 반입신고/
	
	
위험물 일람표 신고
• KL-NET(프리즘 3.0) : https://www.plism.com
• KTNET(ulogishub) : https://www.ulogishub.com  
1
(담당자)민원 신고문서 확인
(수리/반려)
5
민원 신고 완

In [26]:
##### 평가(Critique) agent setup #####

In [27]:
question_groundedness_critique_prompt = """
제공된 맥락과 질문이 주어집니다.
당신의 임무는 제공된 맥락을 가지고 주어진 질문에 명확하게 대답할 수 있는지에 대한 '총 평가' 점수를 제공하는 것입니다.
1부터 5까지의 점수로 평가하며, 1점은 맥락을 가지고 전혀 대답할 수 없음을 의미하고, 5점은 질문을 맥락으로 분명하고 명확하게 대답할 수 있음을 의미합니다.
 
다음과 같이 답변해 주세요:
 
답변:::
평가: (점수를 준 이유에 대한 내용)
총 평가: (1에서 5 사이의 숫자로 점수를 제공)
 
'평가:'와 '총 평가:'에 대한 값이 꼭 포함되어야 합니다.
 
이제 질문과 맥락이 있습니다.
 
질문: {question}\n
맥락: {context}\n
답변::: """
 
question_relevance_critique_prompt = """
질문이 주어집니다.
당신의 임무는 이 질문이 "포트미스(해운항만물류정보시스템) 사이트"를 사용하는 소비자들에게 얼마나 유용한지 대표하는 '총 평가' 점수를 제공하는 것입니다.
1점은 전혀 유용하지 않음을, 5점은 매우 유용함을 의미하는 1에서 5까지의 점수로 평가하세요.

다음과 같이 답변해 주세요:
 
답변:::
평가: (점수를 준 이유에 대한 내용)
총 평가: (1에서 5 사이의 숫자)
 
'평가:'와 '총 평가:'에 대한 값이 꼭 포함되어야 합니다.
 
이제 질문이 있습니다.
 
질문: {question}\n
답변::: """
 
question_standalone_critique_prompt = """
질문이 주어집니다.
당신의 임무는 이 질문이 얼마나 맥락에 독립적인지 나타내는 '총 평가' 점수를 제공하는 것입니다.
1점은 추가 정보가 필요함을 의미하며, 5점은 질문이 스스로 이해가 가능함을 의미합니다.
예를 들어, '맥락에서'나 '문서에서'와 같은 특정 설정을 언급하는 경우 점수는 1이어야 합니다.
질문에 Gradio, Hub, Hugging Face 또는 Space와 같은 기술적 용어나 약어가 포함되어 있어도 상관없지만, 문서를 통해 질문 내용을 이해할 수 있는 운영자에게 명확해야 합니다.
 
예를 들어, "ViT 모델이 어느 체크포인트에서 가져오는가?"는 특정 맥락이 암시되어 있기 때문에 1점을 받아야 합니다.

질문이 주어집니다.
당신의 임무는 이 질문이 얼마나 맥락에 독립적인지 나타내는 '총 평가' 점수를 제공하는 것입니다.
1점은 추가 정보가 필요함을 의미하며, 5점은 질문이 스스로 이해가 가능함을 의미합니다.
예를 들어, '맥락에서'나 '문서에서'와 같은 특정 설정을 언급하는 경우 점수는 1이어야 합니다.
질문에 포트미스 가이드북과 관련된 기술적 용어나 약어가 포함되어 있어도 상관없지만, 문서를 통해 질문 내용을 이해할 수 있는 운영자에게 명확해야 합니다.

예를 들어, "저속운항선박지원신청은 어느 체크포인트에서 가져오는가?"는 특정 맥락이 암시되어 있기 때문에 1점을 받아야 합니다.


다음과 같이 답변해 주세요:
 
답변:::
평가: (점수를 준 이유에 대한 내용)
총 평가: (1에서 5 사이의 숫자)
 
'평가:'와 '총 평가:'에 대한 값이 꼭 포함되어야 합니다.
 
이제 질문이 있습니다.
 
질문: {question}\n
답변::: """

In [38]:
for output in tqdm(outputs):
    evaluations = {
        "groundedness": call_llm(
            question_groundedness_critique_prompt.format(context=output["context"], question=output["question"]),
        ),
        "relevance": call_llm(
            question_relevance_critique_prompt.format(question=output["question"]),
        ),
        "standalone": call_llm(
            question_standalone_critique_prompt.format(question=output["question"]),
        ),
    }
    try:
        for criterion, evaluation in evaluations.items():
            print(f"Evaluating {criterion}: {evaluation}")  # Debugging line
            # '평가:'와 '총 평가:'를 기준으로 분리
            eval_parts = evaluation.split("평가:")
            score_parts = evaluation.split("총 평가:")
            
            if len(eval_parts) > 1 and len(score_parts) > 1:
                eval_text = eval_parts[1].split("총 평가:")[0].strip()
                score = int(score_parts[1].strip())
                output.update(
                    {
                        f"{criterion}_score": score,
                        f"{criterion}_eval": eval_text,
                    }
                )
            else:
                print(f"Unexpected response format for {criterion}: {evaluation}")
    except Exception as e:
        print(f"Error processing {criterion}: {e}")  # Error logging line
        continue

  2%|█▋                                                                                 | 1/49 [00:04<03:21,  4.19s/it]

Evaluating groundedness: 평가: 맥락에서 위험물을 무역항의 수상구역으로 반입할 때의 신고 시기에 대한 명확한 정보가 제공되었습니다.
총 평가: 5


Evaluating relevance: 평가: 포트미스(해운항만물류정보시스템) 사이트는 위험물을 무역항의 수상구역으로 반입할 때 신고 시기에 대한 명확하고 상세한 정보를 제공합니다. 해당 정보를 통해 사용자들은 신고 시기를 정확히 파악할 수 있습니다.

총 평가: 5
Evaluating standalone: 평가: 질문이 구체적이며 명확하게 이해할 수 있습니다.
총 평가: 5


  4%|███▍                                                                               | 2/49 [00:08<03:18,  4.21s/it]

Evaluating groundedness: 답변:::
평가: The information included in the "특별 또는 추가 보안조치 내용" section is related to the special or additional security measures.
총 평가: 5
Evaluating relevance: 평가: "특별 또는 추가 보안조치 내용" 섹션에는 사용자들이 안전하게 정보를 제공하고 전송할 수 있도록 필요한 추가적인 보안 조치에 대한 정보가 포함되어 있습니다.
총 평가: 5
Evaluating standalone: 평가: 질문이 명확하게 구성되어 있으며 추가 정보나 특정 맥락이 필요하지 않습니다.
총 평가: 5

답변: "특별 또는 추가 보안조치 내용" 섹션에는 추가적인 보안 조치나 특별한 보안 관련 정보가 포함되어 있습니다.
Error processing standalone: invalid literal for int() with base 10: '5\n\n답변: "특별 또는 추가 보안조치 내용" 섹션에는 추가적인 보안 조치나 특별한 보안 관련 정보가 포함되어 있습니다.'


  6%|█████                                                                              | 3/49 [00:15<04:07,  5.38s/it]

...

Evaluating groundedness: 평가: PORT-MIS는 전국 무역항에서 선박의 입항, 출항, 항만이용, 항만물류, 해운, 선원, 선박 등의 민원사무 처리 및 정보관리를 지원하는 시스템입니다.
총 평가: 5

Evaluating relevance: 평가: 포트미스(해운항만물류정보시스템)는 무역항에서 화물 추적, 입출항 정보 제공, 물류 정보 관리 등 다양한 업무를 지원합니다.
총 평가: 5
Evaluating standalone: 평가: 질문이 명확하게 제시되어 있으며 추가적인 맥락이나 정보가 필요하지 않습니다.
총 평가: 5

답변: PORT-MIS는 무역항에서 다양한 업무를 지원하는 시스템입니다.
Error processing standalone: invalid literal for int() with base 10: '5\n\n답변: PORT-MIS는 무역항에서 다양한 업무를 지원하는 시스템입니다.'


 98%|████████████████████████████████████████████████████████████████████████████████▎ | 48/49 [03:53<00:04,  4.02s/it]

Evaluating groundedness: 평가: PORT-MIS는 전국 무역항에서 선박의 입항/출항, 항만이용, 항만물류, 해운, 선원, 선박 등의 민원사무 처리 및 정보관리를 지원하는 시스템입니다.
총 평가: 5

Evaluating relevance: 평가: 포트미스(해운항만물류정보시스템)는 무역항에서 화물 추적, 입출항 정보 제공, 물류 정보 관리 등 다양한 업무를 지원합니다.
총 평가: 5
Evaluating standalone: 평가: 질문에서 PORT-MIS가 무역항에서 어떤 업무를 지원하는지에 대한 추가 맥락이 없습니다.
총 평가: 1


100%|██████████████████████████████████████████████████████████████████████████████████| 49/49 [03:56<00:00,  4.84s/it]

Evaluating groundedness: 평가: 제공된 맥락에서 포트미스 이용 환경에서 제공되는 모바일 서비스에 대한 구체적인 정보가 명시적으로 나와 있습니다. 모바일서비스에 대한 설명과 기능들이 명확하게 제시되어 있습니다.
총 평가: 5


Evaluating relevance: 평가: 포트미스의 모바일 서비스는 편리하고 사용하기 쉽습니다. 사용자들이 언제 어디서나 해운항만물류정보를 확인할 수 있어서 유용합니다.
총 평가: 5
Evaluating standalone: 평가: 질문이 명확하게 제공된 맥락에 독립적이며 추가 정보 없이도 이해 가능합니다.
총 평가: 5


In [39]:
import pandas as pd
 
pd.set_option("display.max_colwidth", None)

generated_questions = pd.DataFrame.from_dict(outputs)

print("Evaluation dataset before filtering:")
display(
    generated_questions[
        [
            "question",
            "answer",
            "groundedness_score",
            "relevance_score",
            "standalone_score",
        ]
    ]
)
generated_questions = generated_questions.loc[
    (generated_questions["groundedness_score"] >= 4)
    & (generated_questions["relevance_score"] >= 4)
    & (generated_questions["standalone_score"] >= 4)
]
print("============================================")
print("Final evaluation dataset:")
display(
    generated_questions[
        [
            "question",
            "answer",
            "groundedness_score",
            "relevance_score",
            "standalone_score",
        ]
    ]
)

Evaluation dataset before filtering:


,question,answer,groundedness_score,relevance_score,standalone_score
0,위험물을 무역항의 수상구역으로 반입할 때 신고는 언제 해야 하나요?,위험물을 무역항의 수상구역으로 반입할 때는 반입 24시간 전에 신고해야 합니다.,5.0,5.0,5.0
1,"What information is included in the ""특별 또는 추가 보안조치 내용"" section?",The contents of the special or additional security measures.,5.0,5.0,NaN
2,외항입항신고를 최초로 제출하는 경우 어떤 단계를 거쳐야 하는가?,승무원/승객명부를 담당자(수리)에게 제출해야 합니다.,5.0,NaN,NaN
3,외항선 입항신고에서 선박이 입항 전에 어떤 정보를 신고해야 하는가?,선박이 외항선 입항신고에서 선박이 입항 전에는 입항 예정정보를 신고해야 합니다.,5.0,NaN,NaN
4,해운항만물류정보시스템(PORT-MIS) 운영에 참여하는 공무원과 위탁운영인력의 총 인원은 몇 명인가요?,"공무원 6명, 위탁운영인력 17명으로 총 23명이 참여합니다.",5.0,NaN,NaN
5,PORT-MIS는 무역항에서 어떤 업무를 지원하는 시스템인가요?,"전국 무역항 선박의 입항`출항, 항만이용 및 항만물류, 해운`선원`선박 등의 민원사무 처리 및 정보관리에 관한 업무를 지원하는 시스템입니다.",5.0,5.0,NaN
6,PORT-MIS에서 고지서를 직접 출력할 수 있는 방법은 무엇인가요?,포트미스 접속 후 항만민원 ➡ 조회/발급/사용료 ➡ 사용료 ➡ [고지서 출력(민원인)] 메뉴 선택하여 출력할 고지서를 검색하고 출력대상을 선택한 후 개별 고지서 출력 클릭하여 프린트 또는 pdf 파일로 저장할 수 있습니다.,5.0,5.0,1.0
7,PORT-MIS에서 납부확인서를 출력하는 방법은 무엇인가요?,"PORT-MIS에 접속한 후 항만민원 ➡ 조회/발급/사용료 ➡ 사용료 ➡ [사용료 납부확인서] 메뉴를 선택하여 정보를 조회하고, 원하는 출력대상을 선택한 후 납부확인서를 출력할 수 있습니다.",5.0,5.0,1.0
8,What is the purpose of a PORT-MIS in the context of a hazardous materials declaration form?,A PORT-MIS is used as an appendix in the hazardous materials declaration form.,4.0,4.0,NaN
9,ebXML 민원신고와 EDI 민원신고의 신고방식은 무엇인가요?,"ebXML 민원신고는 민원업체가 직접 신고하는 방식이며, EDI 민원신고는 중계망사업자를 통한 신고 방식입니다.",5.0,NaN,NaN


Final evaluation dataset:


,question,answer,groundedness_score,relevance_score,standalone_score
0,위험물을 무역항의 수상구역으로 반입할 때 신고는 언제 해야 하나요?,위험물을 무역항의 수상구역으로 반입할 때는 반입 24시간 전에 신고해야 합니다.,5.0,5.0,5.0
12,외항입항신고를 최종으로 담당자(수리)와 담당자(반려)가 누구인가요?,"외항입항신고를 최종으로 담당자(수리)는 해당 업무를 처리하는 담당자이며, 담당자(반려)는 해당 업무를 거부하는 담당자입니다.",5.0,4.0,5.0
28,포트미스 이용 환경에서 제공되는 모바일서비스는 무엇인가요?,"포트미스 이용 환경에서 제공되는 모바일서비스는 민원신고 및 신고 결과 조회, 선박에 대한 관제 현황 정보 조회, EDI 및 ebXML 응답문서 조회 기능을 제공합니다.",5.0,4.0,5.0
29,화물료 대납경비 청구 시 수수료는 어떻게 청구되나요?,"대납한 화물료 고지금액의 수수료는 익월에 3%로 청구되며, 보안료는 6.8%로 청구됩니다.",5.0,4.0,5.0
33,포트미스 이용 환경에서 제공되는 모바일 서비스는 무엇인가요?,"포트미스 이용 환경에서 제공되는 모바일 서비스는 민원신고 및 신고 결과 조회, 선박 관제 현황 정보 조회, 그리고 EDI, ebXML 응답문서 조회 기능을 포함합니다.",5.0,5.0,5.0
48,포트미스 이용 환경에서 제공되는 모바일 서비스는 무엇인가요?,"포트미스 이용 환경에서 제공되는 모바일 서비스는 민원신고 및 신고 결과 조회, 선박 관제 현황 정보 조회, 그리고 EDI, ebXML 응답문서 조회 기능을 포함합니다.",5.0,5.0,5.0


In [40]:
##### 검증 데이터셋 기준으로 RAG 결과 획득 #####

In [45]:
#pip install datasets

In [46]:
import datasets
eval_dataset = datasets.Dataset.from_pandas(generated_questions, split="train", preserve_index=False)

In [56]:
import json
from typing import Optional
from langchain.chat_models.base import BaseChatModel
from langchain.vectorstores.base import VectorStore
from transformers import RagTokenizer, RagRetriever, RagSequenceForGeneration

def run_rag_tests(
    eval_dataset: datasets.Dataset,
    llm: BaseChatModel,
    knowledge_index: VectorStore,
    output_file: str,
    reranker: Optional[RagSequenceForGeneration] = None,
    verbose: Optional[bool] = True,
    test_settings: Optional[str] = None,
):
    """주어진 데이터셋에 RAG 테스트를 수행하고 결과를 지정된 출력 파일에 저장합니다."""
    try:  # 기존 결과 로드 시도
        with open(output_file, "r") as f:
            outputs = json.load(f)
    except:
        outputs = []
 
    for example in tqdm(eval_dataset):  # 데이터셋 반복 처리
        question = example["question"]
        if question in [output["question"] for output in outputs]:
            continue
 
        answer, relevant_docs = answer_with_rag(question, llm, knowledge_index, reranker=reranker)
        if verbose:
            print("=======================================================")
            print(f"Question: {question}")
            print(f"Answer: {answer}")
            print(f'True answer: {example["answer"]}')
        result = {
            "question": question,
            "true_answer": example["answer"],
            "source_doc": example["source_doc"],
            "generated_answer": answer,
            "retrieved_docs": [doc for doc in relevant_docs],
        }
        if test_settings:
            result["test_settings"] = test_settings
        outputs.append(result)
 
        with open(output_file, "w") as f:  # 결과 파일 저장
            json.dump(outputs, f)

In [57]:
##### 검증 프롬프트 #####

In [58]:
EVALUATION_PROMPT = """###작업 설명:
지시사항(내부에 입력이 포함될 수 있음), 평가할 응답, 5점을 받은 참조 답안, 그리고 평가 기준을 나타내는 점수 루브릭이 제공됩니다.
1. 주어진 평가 기준에 따라 응답의 질을 엄격하게 평가하는 상세한 피드백을 작성합니다.
2. 피드백 작성 후, 1에서 5 사이의 정수 점수를 작성합니다. 점수 루브릭을 참조해야 합니다.
3. 출력 형식은 다음과 같습니다: \"Feedback: {{평가 기준에 대한 피드백 작성}} [RESULT] {{1에서 5 사이의 정수}}"
4. 다른 서문, 결론, 설명은 생성하지 말고, 출력에 [RESULT]를 반드시 포함하세요.
 
###평가할 지시사항:
{instruction}
 
###평가할 응답:
{response}
 
###참조 답안 (점수 5):
{reference_answer}
 
###점수 루브릭:
[응답이 참조 답안에 기반하여 정확하고 사실적인가?]
점수 1: 응답이 완전히 정확하지 않거나 사실적이지 않습니다.
점수 2: 응답이 대부분 정확하지 않거나 사실적이지 않습니다.
점수 3: 응답이 어느 정도 정확하고 사실적입니다.
점수 4: 응답이 대부분 정확하고 사실적입니다.
점수 5: 응답이 완전히 정확하고 사실적입니다.
 
###피드백:"""
 
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import SystemMessage
 
 
evaluation_prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="당신은 공정한 평가자 언어 모델입니다."),
        HumanMessagePromptTemplate.from_template(EVALUATION_PROMPT),
    ]
)

In [59]:
##### 평가 함수 #####

In [63]:
from langchain.chat_models import ChatOpenAI

eval_chat_model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, api_key="키입력")
evaluator_name = "GPT3.5"
 
def evaluate_answers(
    answer_path: str,
    eval_chat_model: BaseChatModel,
    evaluator_name: str,
    evaluation_prompt_template: ChatPromptTemplate,
) -> None:
    """생성된 답변을 평가합니다. 보다 나은 체크포인팅을 위해 주어진 답변 파일을 현장에서 수정합니다."""
    answers = []
    if os.path.isfile(answer_path):  # 기존 생성물이 있으면 불러옵니다
        answers = json.load(open(answer_path, "r"))
 
    for experiment in tqdm(answers):
        if f"eval_score_{evaluator_name}" in experiment:
            continue
 
        eval_prompt = evaluation_prompt_template.format_messages(
            instruction=experiment["question"],
            response=experiment["generated_answer"],
            reference_answer=experiment["true_answer"],
        )
        eval_result = eval_chat_model.invoke(eval_prompt)
        feedback, score = [item.strip() for item in eval_result.content.split("[RESULT]")]
        experiment[f"eval_score_{evaluator_name}"] = score
        experiment[f"eval_feedback_{evaluator_name}"] = feedback
 
        with open(answer_path, "w") as f:
            json.dump(answers, f)

In [64]:
##### 테스트 실행 #####

In [74]:
...모르겠어요...

SyntaxError: invalid syntax (1897770250.py, line 1)